In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [3]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Users/maxcalabro/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [5]:
len(y2015)

421097

## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas and see what happens off of this kind of naive approach.

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [6]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
12
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [7]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [8]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)



In [11]:
cross_val_score(rfc, X, Y, cv=10)

array([ 0.97972026,  0.97988649,  0.98143003,  0.98171499,  0.95331275,
        0.9733555 ,  0.94528486,  0.98064547,  0.98026456,  0.98047784])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

A lot of these columns are highly correlated to each other, but I'm not sure how to pick out the worst offenders... Let's first get rid of more of the columns with high number of possible outcomes.

In [39]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

In [41]:
dtr = DecisionTreeClassifier(max_depth=6)
cross_val_score(dtr, X, Y, cv=10)

array([ 0.98062264,  0.97988649,  0.98124006,  0.98078887,  0.98019473,
        0.97941107,  0.98009927,  0.9797193 ,  0.97959959,  0.97731915])

In [14]:
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']

In [15]:
X = pd.get_dummies(X)
X = X.dropna(axis=1)

In [16]:
X.shape

(421095, 202)

In [43]:
X_simple = y2015.loc[:,['funded_amnt', 'total_pymnt', 'last_pymnt_amnt', 'delinq_amnt', 'home_ownership', 'verification_status', 'mths_since_last_delinq']]

X_simple = pd.get_dummies(X_simple)
X_simple = X_simple.dropna(axis=1)

In [44]:
cross_val_score(rfc, X_simple, Y, cv=10)

array([ 0.73534231,  0.92201563,  0.9118995 ,  0.91199449,  0.90771788,
        0.90465448,  0.89721912,  0.90629082,  0.91319733,  0.9084216 ])

That's right around around 90% accuracy, although the .73 group suggests we're missing something. But we only have 7 of the original variables, which is pretty cool.

In [18]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE

In [19]:
pca = PCA(n_components=10)
pca.fit(X)
X_pca = pca.transform(X)

In [20]:
cross_val_score(rfc, X_pca, Y, cv=10)

array([ 0.69639762,  0.74268006,  0.73662463,  0.75407851,  0.6941819 ,
        0.64602232,  0.5323565 ,  0.52494716,  0.54437504,  0.57395621])

In [42]:
dtr = DecisionTreeClassifier(max_depth=6)
cross_val_score(dtr, X_pca, Y, cv=10)

array([ 0.7001496 ,  0.8991    ,  0.87250362,  0.8873691 ,  0.85699359,
        0.78185704,  0.81177421,  0.49818329,  0.84919372,  0.55688025])

Yikes - Maybe could have expected PCA to not work so well since there were so many dummy variables. 

In [ ]:
rfe = RFE(rfc, n_features_to_select=10)
rfe.fit(X,Y)

Okay -- that was killing my battery and making my computer hot. Let's try SelectKBest and look at features from RFC instead.

In [23]:
from sklearn.feature_selection import SelectKBest

In [25]:
skb = SelectKBest(k=15)
skb.fit(X,Y)
X_skb = skb.transform(X)

/Users/maxcalabro/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [26 98] are constant.
  UserWarning)
/Users/maxcalabro/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [26]:
cross_val_score(rfc, X_skb, Y, cv=10)

array([ 0.95020303,  0.95711334,  0.96402365,  0.96908171,  0.94806459,
        0.96236048,  0.95991356,  0.96832031,  0.96444772,  0.97375671])

In [43]:
dtr = DecisionTreeClassifier(max_depth=6)
cross_val_score(dtr, X_skb, Y, cv=10)

array([ 0.97093396,  0.97112393,  0.97129016,  0.97186008,  0.97057706,
        0.97147946,  0.97112256,  0.97057636,  0.96986249,  0.96919679])

Hey that's great! And what about using feature importances from RFC?

In [27]:
rfc.fit(X,Y)
importance = rfc.feature_importances_

In [37]:
top_feat = X.columns[importance.argsort()[-6:]]

In [38]:
cross_val_score(rfc, X[top_feat], Y, cv=10)

array([ 0.93956448,  0.93386526,  0.93885208,  0.94647479,  0.9124436 ,
        0.94103538,  0.93656938,  0.94497613,  0.92773173,  0.95729825])

In [44]:
dtr = DecisionTreeClassifier(max_depth=6)
cross_val_score(dtr, X[top_feat], Y, cv=10)

array([ 0.95594975,  0.95445371,  0.95542732,  0.95585476,  0.95279031,
        0.95651864,  0.95528272,  0.95435655,  0.95478187,  0.95437705])

Success! 94% accuracy with only six features. 

It's interesting that the decision tree is working better than random forest. Shouldn't it be the other way around? 